In [ ]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Using TensorFlow backend.


In [ ]:
#import data
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

#importing my own data

from google.colab import files
path_to_file = list(files.upload().keys())[0]


Saving dave_chappelle_transcripts.txt to dave_chappelle_transcripts.txt


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 340030 characters


In [ ]:
#data preprocessing
#encode the text into integers (characters)

vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

# function converting integer back to text
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

Sometimes, th


In [ ]:
# Create training examples
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets ; create a stream of characters
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# use batch method to turn stream of characters into batches
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

#splitting the data
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
# make training batches
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# build model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23296     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 91)            93275     
Total params: 5,363,547
Trainable params: 5,363,547
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# creating a loss function

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
#compile the model
model.compile(optimizer='adam', loss=loss)

In [ ]:
#create checkpoints: saves checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
#training the model
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
52/52 [==============================] - 4s 72ms/step - loss: 3.1922
Epoch 2/100
52/52 [==============================] - 4s 73ms/step - loss: 2.4972
Epoch 3/100
52/52 [==============================] - 4s 74ms/step - loss: 2.1769
Epoch 4/100
52/52 [==============================] - 4s 74ms/step - loss: 1.9592
Epoch 5/100
52/52 [==============================] - 4s 74ms/step - loss: 1.8043
Epoch 6/100
52/52 [==============================] - 4s 75ms/step - loss: 1.6869
Epoch 7/100
52/52 [==============================] - 4s 76ms/step - loss: 1.5955
Epoch 8/100
52/52 [==============================] - 4s 75ms/step - loss: 1.5170
Epoch 9/100
52/52 [==============================] - 4s 74ms/step - loss: 1.4508
Epoch 10/100
52/52 [==============================] - 4s 74ms/step - loss: 1.3954
Epoch 11/100
52/52 [==============================] - 4s 73ms/step - loss: 1.3428
Epoch 12/100
52/52 [==============================] - 4s 74ms/step - loss: 1.2979
Epoch 13/100
52/52 [=====

In [ ]:
#load and rebuild model to pass in input samples of batch size 1
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

#find the latest checkpoint that stores the models weights
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
#[1 = dimension 1, None = don't know what size of the input it will be]

In [ ]:
#text generating function by tensorflow
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 8000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0) # places input_eval list into a nested list [] --> [[]], as model expects this as input

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.?
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0) # removes outer [], [[]] --> []

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: thank
thank I’m gonna do in this day and age?” You know what I mean? Cause I was doing some “My This will get our grass a decision?” And when the mom came in and she didn’t say it offended her. What she said is, she said, “I am uncomfortable with that word?” And yet, mine. Shit was my shoulder right now. ”And this guy’s balls are as smells like weed. He was speeding. This man isn the beginning of the back of the bus after you again, Dave Chappelle.” And I looked at my friend like they were very dravorch. Who if he’s drink. They want drink. They don’t want all them vitamins, man. There was a big dance coming? I said “has a white dude walking across my property, ent, the racism down there is just here. He also men go through just for the rent!” [laughing] “Hello? Dan’t want to talk about it. The tape. But then Kevin said, “Fuck it, I quit.” And then he went down like pancake butter. He was higher when stay morthfull decisions all the time. You don’t wanna get piss